In [61]:
from newspaper import Article
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
import time as tm
import re
from bs4 import BeautifulSoup

In [62]:
cleaned = pd.read_csv('cleaned1.csv')

In [65]:
article = Article('https://wayback.archive-it.org/12706/20200113151841/https://newbrunswicktoday.com/2019/06/12/under-pressure-rutgers-releases-rupd-and-nbpd-dispatch-logs/')
#article.html = str(f.read(), 'utf-8')
article.download()
article.parse()
print(article.authors)
print(article.publish_date)
print(article.title)
print('\n')
print(article.text)

['Charlie Kratovil', 'About The Writer', 'More Writer', 'November', 'On', 'In', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav.Molongui-Author-Box-Tabs-Top Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav Label Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Input Id', "'Mab-Tab-Profile-'"]
2019-06-12 00:00:00
Under Pressure, Rutgers Releases RUPD and NBPD Dispatch Logs


NEW BRUNSWICK, NJ—Rutgers University released 1,272 pages of police records after being served with an amended complaint in a public records lawsuit filed by this reporter.

The Computer Aided Dispatch (CAD) logs list each and every call that the city's two major police departments responded to between February 1 and May 31, plus other instances where officers were dispatched.

After eighteen weeks of stalling, Rutgers released the records about 30 hours after being served with the amended complaint.

Then, the following day, Rutgers released an additional 299 pages consis

In [11]:
chosen.date[0]

'2019-10-08'

In [87]:

# this creates and sets all the new variables to None
chosen['cleaned'] = None
chosen['title'] = None
chosen['published'] = None
chosen['author'] = None
chosen['date'] = pd.to_datetime(chosen['date'])

# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def get_html(collection_id, data):
    ctr=0

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for index, row in data.iterrows():

        ctr+=1
        # if ctr > 10: break

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + row.url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:
    
            if dt.strptime(a.text, '%b %d, %Y') == row.date:
                
                try:
                    article = Article('https:' + a.attrs['href'])
                    article.download()
                    article.parse()
                except:
                    print("can't proces")

                try:
                    data.loc[index, 'title'] = article.title
                    if len(article.authors) > 0: 
                        data.loc[index, 'author'] = article.authors[0]
                    data.loc[index, 'published'] = article.publish_date
                    data.loc[index, 'text'] = article.text
                    data.loc[index, 'cleaned'] = 1
                except:
                    data.loc[index, 'cleaned'] = 0

    return data

In [88]:
start = tm.time()
new_parse = get_html(12706, chosen)
span = tm.time() - start
print('completed in ', span, ' seconds')

can't proces
can't proces
C:\Users\matth\anaconda3\envs\holcombEnv\lib\site-packages\PIL\TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 26. Skipping tag 42034
  "Possibly corrupt EXIF data.  "
C:\Users\matth\anaconda3\envs\holcombEnv\lib\site-packages\PIL\TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19 bytes but only got 0. Skipping tag 42036
  "Possibly corrupt EXIF data.  "
can't proces
C:\Users\matth\anaconda3\envs\holcombEnv\lib\site-packages\PIL\TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 918 bytes but only got 314. Skipping tag 37500
  "Possibly corrupt EXIF data.  "
C:\Users\matth\anaconda3\envs\holcombEnv\lib\site-packages\PIL\TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  "Possibly corrupt EXIF data.  "
C:\Users\matth\anaconda3\envs\holcombEnv\lib\site-packages

In [ ]:
new_parse.to_csv('test_parsing.csv')